# 🔬 YOLOv10 Training - TBX11K Tuberculosis Detection 
## CSE475 Machine Learning Lab Assignment

---

**Student:** Shahriar Khan , Rifah Tamannah , Khalid Mahmud Joy , Tanvir Rahman
**Institution:** East West University  
**Model:** YOLOv10n (Nano)  
**Dataset:** TBX11K Small Dataset (800 images total)  
**Training Epochs:** 30  
**Optimization:** Configured for small dataset with aggressive augmentation

---

### 📋 Notebook Overview

This notebook trains **YOLOv10n** model for tuberculosis detection with :
- ✅ AGGRESSIVE augmentation 
- ✅ Larger image size 
- ✅ Smaller batch size 
- ✅ Conservative learning rate 
- ✅ Strong regularization
- ✅ Comprehensive visualizations
- ✅ Training curves and metrics
- ✅ Confusion matrix analysis
- ✅ Sample predictions

### ⚠️ Dataset 
- **Training:** 600 images 
- **Validation:** 200 images
- **Total:** 800 images 




## 📦 Section 1: Environment Setup

In [ ]:
# Installation cell - Run ONCE, then RESTART kernel
print("🔧 Installing compatible packages for Kaggle...")
print("=" * 80)

# Fix NumPy/Matplotlib compatibility
!pip install -q "numpy<2.0" --force-reinstall

# Fix OpenCV compatibility
!pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python 2>/dev/null
!pip install -q opencv-python-headless==4.8.1.78

# Install YOLO
!pip install -q --no-deps ultralytics
!pip install -q pillow tqdm pyyaml

print("=" * 80)
print("✅ Installation complete!")
print("⚠️  RESTART KERNEL NOW: Run → Restart Session")
print("=" * 80)

In [ ]:
# Core Libraries
import os
import sys
import json
import time
import random
import warnings
from pathlib import Path
from datetime import datetime

# Data Processing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

# Computer Vision
import cv2
from PIL import Image

# Deep Learning
import torch
from ultralytics import YOLO

# Warnings
warnings.filterwarnings('ignore')

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Display settings
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## ⚙️ Section 2: Configuration

In [ ]:
class YOLOv10Config:
    """Configuration for YOLOv10 training on TBX11K dataset - OPTIMIZED FOR SMALL DATASET"""
    
    # ========== DATASET PATHS (KAGGLE OPTIMIZED) ==========
    # Update this to match your Kaggle dataset name after upload
    DATASET_NAME = 'tbx11k-small-balanced'  # Change this to your uploaded dataset name
    DATASET_PATH = f'/kaggle/input/{DATASET_NAME}'
    DATA_YAML = f'{DATASET_PATH}/data.yaml'
    
    # ========== OUTPUT PATHS ==========
    OUTPUT_DIR = Path('/kaggle/working')
    MODEL_DIR = OUTPUT_DIR / 'yolov10_model'
    PLOTS_DIR = OUTPUT_DIR / 'yolov10_plots'
    RESULTS_DIR = OUTPUT_DIR / 'yolov10_results'
    
    # Create directories
    for directory in [MODEL_DIR, PLOTS_DIR, RESULTS_DIR]:
        directory.mkdir(parents=True, exist_ok=True)
    
    # ========== MODEL CONFIGURATION ==========
    MODEL_NAME = 'YOLOv10n'
    MODEL_WEIGHTS = 'yolov10n.pt'
    
    # ========== TRAINING HYPERPARAMETERS (OPTIMIZED FOR SMALL DATA) ==========
    IMG_SIZE = 640  # Increased from 512 for more detail
    BATCH_SIZE = 8  # Reduced from 16 for more gradient updates
    EPOCHS = 30     # Reduced from 150 for faster training (your request)
    PATIENCE = 15   # Adjusted proportionally
    WORKERS = 0     # Avoid multiprocessing issues
    DEVICE = 0
    
    # ========== OPTIMIZER SETTINGS (CONSERVATIVE FOR SMALL DATA) ==========
    OPTIMIZER = 'AdamW'
    LR0 = 0.0005    # Reduced from 0.001 for gentler learning
    LRF = 0.005     # Gentler decay
    MOMENTUM = 0.937
    WEIGHT_DECAY = 0.001  # Increased regularization
    WARMUP_EPOCHS = 5     # Longer warmup
    WARMUP_MOMENTUM = 0.8
    WARMUP_BIAS_LR = 0.1
    
    # ========== LOSS WEIGHTS ==========
    BOX = 7.5
    CLS = 1.5  # Increased for better class distinction
    DFL = 1.5
    
    # ========== AGGRESSIVE AUGMENTATION (KEY FOR SMALL DATASETS!) ==========
    DEGREES = 25.0      # Increased rotation range
    TRANSLATE = 0.2     # Increased translation
    SCALE = 0.5         # Increased scaling variation
    SHEAR = 10.0        # Increased shearing
    PERSPECTIVE = 0.001
    FLIPUD = 0.0        # No vertical flip for X-rays
    FLIPLR = 0.5        # Horizontal flip
    MOSAIC = 1.0        # ALWAYS use mosaic (combines 4 images)
    MIXUP = 0.3         # Increased from 0.15
    COPY_PASTE = 0.3    # Increased from 0.1 - copies TB lesions
    HSV_H = 0.0         # No hue (grayscale X-rays)
    HSV_S = 0.0         # No saturation
    HSV_V = 0.6         # Increased brightness variation
    ERASING = 0.5       # Increased random erasing
    
    # ========== REGULARIZATION ==========
    DROPOUT = 0.3
    LABEL_SMOOTHING = 0.1
    
    # ========== INFERENCE ==========
    CONF_THRESHOLD = 0.20  # Lowered from 0.25 to increase recall
    IOU_THRESHOLD = 0.45
    
    # ========== DATASET INFO ==========
    NUM_CLASSES = 3
    CLASS_NAMES = {
        0: 'Active Tuberculosis',
        1: 'Obsolete Pulmonary TB',
        2: 'Pulmonary Tuberculosis'
    }
    
    # ========== VISUALIZATION ==========
    DPI = 150
    FIGSIZE = (15, 10)

config = YOLOv10Config()

print("=" * 80)
print("CONFIGURATION - OPTIMIZED FOR SMALL DATASET (800 images)")
print("=" * 80)
print(f"Dataset: {config.DATASET_PATH}")
print(f"Data YAML: {config.DATA_YAML}")
print(f"Model: {config.MODEL_NAME}")
print(f"Image Size: {config.IMG_SIZE}x{config.IMG_SIZE} (increased from 512)")
print(f"Batch Size: {config.BATCH_SIZE} (reduced for more updates)")
print(f"Epochs: {config.EPOCHS}")
print(f"Patience: {config.PATIENCE}")
print(f"Classes: {config.NUM_CLASSES}")
print(f"Learning Rate: {config.LR0} (conservative)")
print(f"Augmentation: AGGRESSIVE (Mosaic=1.0, MixUp=0.3, CopyPaste=0.3)")
print(f"Output: {config.OUTPUT_DIR}")
print("=" * 80)
print("KEY CHANGES FOR SMALL DATASET:")
print("  - Batch size: 16 -> 8 (more gradient updates)")
print("  - Image size: 512 -> 640 (capture more detail)")
print("  - LR: 0.001 -> 0.0005 (gentler learning)")
print("  - Mosaic: 0.8 -> 1.0 (always combine 4 images)")
print("  - MixUp: 0.15 -> 0.3 (more mixing)")
print("  - CopyPaste: 0.1 -> 0.3 (copy TB lesions)")
print("  - Rotation: +/-15 deg -> +/-25 deg (more variation)")
print("  - Scale: +/-30% -> +/-50% (diverse sizes)")
print("  - Brightness: 40% -> 60% (strong variation)")
print("  - Erasing: 40% -> 50% (simulate occlusions)")
print("  - Conf threshold: 0.25 -> 0.20 (increase recall)")
print("=" * 80)
print("IMPORTANT: Update DATASET_NAME in config to match your Kaggle dataset!")
print("=" * 80)

## 📊 Section 3: Dataset Verification

In [ ]:
# Verify dataset structure
print("🔍 Verifying dataset structure...\n")

dataset_path = Path(config.DATASET_PATH)

# Check main directories
train_img_dir = dataset_path / 'images' / 'train'
train_lbl_dir = dataset_path / 'labels' / 'train'
val_img_dir = dataset_path / 'images' / 'val'
val_lbl_dir = dataset_path / 'labels' / 'val'

# Count files
train_images = list(train_img_dir.glob('*.png')) if train_img_dir.exists() else []
train_labels = list(train_lbl_dir.glob('*.txt')) if train_lbl_dir.exists() else []
val_images = list(val_img_dir.glob('*.png')) if val_img_dir.exists() else []
val_labels = list(val_lbl_dir.glob('*.txt')) if val_lbl_dir.exists() else []

print("📂 Dataset Structure:")
print(f"  ├─ Training Images: {len(train_images)}")
print(f"  ├─ Training Labels: {len(train_labels)}")
print(f"  ├─ Validation Images: {len(val_images)}")
print(f"  └─ Validation Labels: {len(val_labels)}")

# Check data.yaml
data_yaml_path = Path(config.DATA_YAML)
if data_yaml_path.exists():
    print(f"\n✅ data.yaml found: {data_yaml_path}")
    with open(data_yaml_path, 'r') as f:
        print("\n📄 data.yaml content:")
        print(f.read())
else:
    print(f"\n⚠️  data.yaml not found at: {data_yaml_path}")

## 🚀 Section 4: Model Training

In [ ]:
print("=" * 80)
print("🚀 STARTING YOLOV10 TRAINING - OPTIMIZED FOR SMALL DATASET")
print("=" * 80)

# Initialize model
print(f"\n📥 Loading pretrained weights: {config.MODEL_WEIGHTS}")
model = YOLO(config.MODEL_WEIGHTS)

# Training arguments
train_args = {
    'data': str(config.DATA_YAML),
    'epochs': config.EPOCHS,
    'imgsz': config.IMG_SIZE,
    'batch': config.BATCH_SIZE,
    'device': config.DEVICE,
    'workers': config.WORKERS,
    'patience': config.PATIENCE,
    'save': True,
    'save_period': 10,
    'cache': False,
    'project': str(config.MODEL_DIR),
    'name': 'train',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': config.OPTIMIZER,
    'verbose': True,
    'seed': 42,
    'deterministic': False,
    'single_cls': False,
    'rect': False,
    'cos_lr': True,
    'close_mosaic': 5,
    'resume': False,
    'amp': False,
    'fraction': 1.0,
    'profile': False,
    'freeze': None,
    'plots': True,
    
    # Hyperparameters (OPTIMIZED)
    'lr0': config.LR0,
    'lrf': config.LRF,
    'momentum': config.MOMENTUM,
    'weight_decay': config.WEIGHT_DECAY,
    'warmup_epochs': config.WARMUP_EPOCHS,
    'warmup_momentum': config.WARMUP_MOMENTUM,
    'warmup_bias_lr': config.WARMUP_BIAS_LR,
    'box': config.BOX,
    'cls': config.CLS,
    'dfl': config.DFL,
    'dropout': config.DROPOUT,
    'label_smoothing': config.LABEL_SMOOTHING,
    
    # AGGRESSIVE Augmentation (KEY FOR SMALL DATA!)
    'hsv_h': config.HSV_H,
    'hsv_s': config.HSV_S,
    'hsv_v': config.HSV_V,
    'degrees': config.DEGREES,
    'translate': config.TRANSLATE,
    'scale': config.SCALE,
    'shear': config.SHEAR,
    'perspective': config.PERSPECTIVE,
    'flipud': config.FLIPUD,
    'fliplr': config.FLIPLR,
    'mosaic': config.MOSAIC,
    'mixup': config.MIXUP,
    'copy_paste': config.COPY_PASTE,
    'erasing': config.ERASING,
}

print("\n📋 Training Configuration (OPTIMIZED FOR SMALL DATASET):")
print(f"  • Epochs: {config.EPOCHS} (reduced for time constraint)")
print(f"  • Batch Size: {config.BATCH_SIZE} ⬇️ (more updates)")
print(f"  • Image Size: {config.IMG_SIZE} ⬆️ (more detail)")
print(f"  • Optimizer: {config.OPTIMIZER}")
print(f"  • Learning Rate: {config.LR0} ⬇️ (conservative)")
print(f"  • Weight Decay: {config.WEIGHT_DECAY} ⬆️ (regularization)")
print(f"  • Dropout: {config.DROPOUT}")
print(f"  • Label Smoothing: {config.LABEL_SMOOTHING}")
print(f"\n🎨 AGGRESSIVE Augmentation (maximize 800 images):")
print(f"  • Mosaic: {config.MOSAIC} (ALWAYS combine 4 images)")
print(f"  • MixUp: {config.MIXUP}")
print(f"  • Copy-Paste: {config.COPY_PASTE} (copy TB lesions)")
print(f"  • Rotation: ±{config.DEGREES}°")
print(f"  • Translation: {config.TRANSLATE*100}%")
print(f"  • Scale: ±{config.SCALE*100}%")
print(f"  • Shear: ±{config.SHEAR}°")
print(f"  • Brightness: {config.HSV_V*100}% variation")
print(f"  • Random Erasing: {config.ERASING*100}%")
print(f"  • Horizontal Flip: {config.FLIPLR*100}%")

print("\n⏳ Training started... This will take approximately 15-20 minutes.\n")
print("🎯 Expected improvement over baseline:")
print("   mAP@0.5: 0.24 → 0.35-0.45 (+46-88%)")
print("   Precision: 0.30 → 0.45-0.55")
print("   Recall: 0.22 → 0.35-0.45\n")

# Start training
start_time = time.time()
results = model.train(**train_args)
training_time = time.time() - start_time

print("\n" + "=" * 80)
print("✅ TRAINING COMPLETED!")
print("=" * 80)
print(f"⏱️  Training Time: {training_time/60:.2f} minutes ({training_time/3600:.2f} hours)")
print(f"💾 Model saved to: {config.MODEL_DIR / 'train' / 'weights' / 'best.pt'}")

## 📈 Section 5: Validation & Metrics

In [ ]:
print("=" * 80)
print("📊 VALIDATING YOLOV10 MODEL")
print("=" * 80)

# Load best model
best_model_path = config.MODEL_DIR / 'train' / 'weights' / 'best.pt'
best_model = YOLO(str(best_model_path))

print(f"\n📥 Loaded best model: {best_model_path}")

# Run validation
print("\n⏳ Running validation...\n")
val_results = best_model.val(
    data=str(config.DATA_YAML),
    split='val',
    imgsz=config.IMG_SIZE,
    batch=config.BATCH_SIZE,
    conf=config.CONF_THRESHOLD,
    iou=config.IOU_THRESHOLD,
    device=config.DEVICE,
    workers=config.WORKERS,
    plots=True,
    save_json=True,
    project=str(config.RESULTS_DIR),
    name='validation',
    exist_ok=True
)

# Extract metrics
print("\n" + "=" * 80)
print("📈 VALIDATION RESULTS")
print("=" * 80)
print(f"  • mAP@0.5:     {val_results.box.map50:.4f}")
print(f"  • mAP@0.5:0.95: {val_results.box.map:.4f}")
print(f"  • Precision:    {val_results.box.mp:.4f}")
print(f"  • Recall:       {val_results.box.mr:.4f}")
print(f"  • Fitness:      {val_results.fitness:.4f}")
print("=" * 80)

# Save metrics to JSON
metrics = {
    'model': config.MODEL_NAME,
    'epochs': config.EPOCHS,
    'training_time_minutes': training_time / 60,
    'mAP50': float(val_results.box.map50),
    'mAP50_95': float(val_results.box.map),
    'precision': float(val_results.box.mp),
    'recall': float(val_results.box.mr),
    'fitness': float(val_results.fitness),
}

metrics_file = config.RESULTS_DIR / 'yolov10_metrics.json'
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\n💾 Metrics saved to: {metrics_file}")

## 📊 Section 6: Training Curves Visualization

In [ ]:
# Read training results
results_csv = config.MODEL_DIR / 'train' / 'results.csv'

if results_csv.exists():
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()
    
    # Create comprehensive training curves
    fig, axes = plt.subplots(3, 3, figsize=(20, 15))
    fig.suptitle('YOLOv10 Training Curves - Complete Analysis', fontsize=18, fontweight='bold')
    
    epochs = df['epoch'] if 'epoch' in df.columns else range(len(df))
    
    # Plot 1: mAP@0.5
    if 'metrics/mAP50(B)' in df.columns:
        axes[0, 0].plot(epochs, df['metrics/mAP50(B)'], linewidth=2.5, color='blue', label='mAP@0.5')
        axes[0, 0].fill_between(epochs, df['metrics/mAP50(B)'], alpha=0.3, color='blue')
        axes[0, 0].set_title('mAP@0.5', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('mAP@0.5')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].legend()
    
    # Plot 2: mAP@0.5:0.95
    if 'metrics/mAP50-95(B)' in df.columns:
        axes[0, 1].plot(epochs, df['metrics/mAP50-95(B)'], linewidth=2.5, color='green', label='mAP@0.5:0.95')
        axes[0, 1].fill_between(epochs, df['metrics/mAP50-95(B)'], alpha=0.3, color='green')
        axes[0, 1].set_title('mAP@0.5:0.95', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('mAP@0.5:0.95')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].legend()
    
    # Plot 3: Precision
    if 'metrics/precision(B)' in df.columns:
        axes[0, 2].plot(epochs, df['metrics/precision(B)'], linewidth=2.5, color='orange', label='Precision')
        axes[0, 2].fill_between(epochs, df['metrics/precision(B)'], alpha=0.3, color='orange')
        axes[0, 2].set_title('Precision', fontsize=14, fontweight='bold')
        axes[0, 2].set_xlabel('Epoch')
        axes[0, 2].set_ylabel('Precision')
        axes[0, 2].grid(True, alpha=0.3)
        axes[0, 2].legend()
    
    # Plot 4: Recall
    if 'metrics/recall(B)' in df.columns:
        axes[1, 0].plot(epochs, df['metrics/recall(B)'], linewidth=2.5, color='red', label='Recall')
        axes[1, 0].fill_between(epochs, df['metrics/recall(B)'], alpha=0.3, color='red')
        axes[1, 0].set_title('Recall', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Recall')
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].legend()
    
    # Plot 5: Box Loss
    if 'train/box_loss' in df.columns and 'val/box_loss' in df.columns:
        axes[1, 1].plot(epochs, df['train/box_loss'], linewidth=2, label='Train', color='blue')
        axes[1, 1].plot(epochs, df['val/box_loss'], linewidth=2, label='Val', color='red')
        axes[1, 1].set_title('Box Loss', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].legend()
    
    # Plot 6: Class Loss
    if 'train/cls_loss' in df.columns and 'val/cls_loss' in df.columns:
        axes[1, 2].plot(epochs, df['train/cls_loss'], linewidth=2, label='Train', color='blue')
        axes[1, 2].plot(epochs, df['val/cls_loss'], linewidth=2, label='Val', color='red')
        axes[1, 2].set_title('Classification Loss', fontsize=14, fontweight='bold')
        axes[1, 2].set_xlabel('Epoch')
        axes[1, 2].set_ylabel('Loss')
        axes[1, 2].grid(True, alpha=0.3)
        axes[1, 2].legend()
    
    # Plot 7: DFL Loss
    if 'train/dfl_loss' in df.columns and 'val/dfl_loss' in df.columns:
        axes[2, 0].plot(epochs, df['train/dfl_loss'], linewidth=2, label='Train', color='blue')
        axes[2, 0].plot(epochs, df['val/dfl_loss'], linewidth=2, label='Val', color='red')
        axes[2, 0].set_title('DFL Loss', fontsize=14, fontweight='bold')
        axes[2, 0].set_xlabel('Epoch')
        axes[2, 0].set_ylabel('Loss')
        axes[2, 0].grid(True, alpha=0.3)
        axes[2, 0].legend()
    
    # Plot 8: F1 Score (calculated)
    if 'metrics/precision(B)' in df.columns and 'metrics/recall(B)' in df.columns:
        precision = df['metrics/precision(B)']
        recall = df['metrics/recall(B)']
        f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
        axes[2, 1].plot(epochs, f1, linewidth=2.5, color='purple', label='F1 Score')
        axes[2, 1].fill_between(epochs, f1, alpha=0.3, color='purple')
        axes[2, 1].set_title('F1 Score', fontsize=14, fontweight='bold')
        axes[2, 1].set_xlabel('Epoch')
        axes[2, 1].set_ylabel('F1 Score')
        axes[2, 1].grid(True, alpha=0.3)
        axes[2, 1].legend()
    
    # Plot 9: Learning Rate
    if 'lr/pg0' in df.columns:
        axes[2, 2].plot(epochs, df['lr/pg0'], linewidth=2, color='brown', label='Learning Rate')
        axes[2, 2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
        axes[2, 2].set_xlabel('Epoch')
        axes[2, 2].set_ylabel('Learning Rate')
        axes[2, 2].grid(True, alpha=0.3)
        axes[2, 2].legend()
    
    plt.tight_layout()
    save_path = config.PLOTS_DIR / 'training_curves.png'
    plt.savefig(save_path, dpi=config.DPI, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Training curves saved to: {save_path}")
else:
    print(f"⚠️  Results CSV not found at: {results_csv}")

## 🎯 Section 7: Confusion Matrix

In [ ]:
# Check for confusion matrix
confusion_matrix_path = config.RESULTS_DIR / 'validation' / 'confusion_matrix.png'

if confusion_matrix_path.exists():
    print("📊 Displaying Confusion Matrix\n")
    
    img = Image.open(confusion_matrix_path)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('YOLOv10 - Confusion Matrix (Normalized)', fontsize=16, fontweight='bold', pad=20)
    
    plt.tight_layout()
    save_path = config.PLOTS_DIR / 'confusion_matrix_display.png'
    plt.savefig(save_path, dpi=config.DPI, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Confusion matrix saved to: {save_path}")
else:
    print(f"⚠️  Confusion matrix not found at: {confusion_matrix_path}")

## 🖼️ Section 8: Sample Predictions

In [ ]:
print("=" * 80)
print("GENERATING SAMPLE PREDICTIONS")
print("=" * 80)

# Get validation images with labels
val_img_dir = Path(config.DATASET_PATH) / 'images' / 'val'
val_lbl_dir = Path(config.DATASET_PATH) / 'labels' / 'val'

# Check if directory exists
if not val_img_dir.exists():
    print(f"ERROR: Validation image directory not found: {val_img_dir}")
    print("Please check your dataset path configuration.")
else:
    val_images_with_labels = []
    for img_path in val_img_dir.glob('*.png'):
        label_path = val_lbl_dir / f"{img_path.stem}.txt"
        if label_path.exists() and label_path.stat().st_size > 0:
            val_images_with_labels.append(img_path)

    if len(val_images_with_labels) == 0:
        print("WARNING: No validation images with labels found!")
        print(f"  - Checked directory: {val_img_dir}")
        print(f"  - Total .png files: {len(list(val_img_dir.glob('*.png')))}")
        print("  - Images with non-empty labels: 0")
    else:
        # Select random samples
        num_samples = min(9, len(val_images_with_labels))
        selected_samples = random.sample(val_images_with_labels, num_samples)

        print(f"\nGenerating predictions for {num_samples} samples...\n")

        # Create prediction grid
        fig, axes = plt.subplots(3, 3, figsize=(20, 20))
        axes = axes.flatten()
        fig.suptitle('YOLOv10 - Sample Predictions on Validation Set', fontsize=18, fontweight='bold')

        for idx, img_path in enumerate(selected_samples):
            # Run prediction
            results = best_model.predict(
                source=str(img_path),
                conf=config.CONF_THRESHOLD,
                iou=config.IOU_THRESHOLD,
                imgsz=config.IMG_SIZE,
                device=config.DEVICE,
                verbose=False
            )
            
            # Get annotated image
            annotated = results[0].plot()
            annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
            
            # Display
            axes[idx].imshow(annotated_rgb)
            num_detections = len(results[0].boxes)
            axes[idx].set_title(f'{img_path.stem}\n({num_detections} detections)', 
                               fontsize=11, fontweight='bold')
            axes[idx].axis('off')

        plt.tight_layout()
        save_path = config.PLOTS_DIR / 'sample_predictions.png'
        plt.savefig(save_path, dpi=config.DPI, bbox_inches='tight')
        plt.show()

        print(f"Sample predictions saved to: {save_path}")

## 📊 Section 9: Metrics Summary Table

## 🎨 Section 8.5: Enhanced Detection Visualizations

In [ ]:
# ============================================================================
# VISUALIZATION 1: Prediction Grid with ALL Images (Even Without Detections)
# ============================================================================
print("=" * 80)
print("CREATING COMPREHENSIVE PREDICTION VISUALIZATIONS")
print("=" * 80)

# Get ALL validation images
val_img_dir = Path(config.DATASET_PATH) / 'images' / 'val'

# Check if directory exists
if not val_img_dir.exists():
    print(f"ERROR: Validation directory not found: {val_img_dir}")
    print("Please verify your DATASET_PATH configuration.")
    all_val_images = []
else:
    all_val_images = list(val_img_dir.glob('*.png'))

if len(all_val_images) == 0:
    print("ERROR: No validation images found!")
    print(f"  - Checked directory: {val_img_dir}")
    print(f"  - Looking for: *.png files")
    print("\nPossible solutions:")
    print("  1. Check if DATASET_NAME in config matches your Kaggle dataset")
    print("  2. Verify dataset structure: images/val/ folder exists")
    print("  3. Ensure images have .png extension")
else:
    # Select 9 random images (regardless of labels)
    num_samples = min(9, len(all_val_images))
    selected_samples = random.sample(all_val_images, num_samples)
    
    print(f"\nGenerating predictions for {num_samples} validation images...\n")
    
    # Create prediction grid
    fig, axes = plt.subplots(3, 3, figsize=(20, 20))
    axes = axes.flatten()
    fig.suptitle('YOLOv10 - Sample Predictions on Validation Set', fontsize=18, fontweight='bold', y=0.995)
    
    for idx, img_path in enumerate(selected_samples):
        try:
            # Run prediction
            results = best_model.predict(
                source=str(img_path),
                conf=config.CONF_THRESHOLD,
                iou=config.IOU_THRESHOLD,
                imgsz=config.IMG_SIZE,
                device=config.DEVICE,
                verbose=False
            )
            
            # Get annotated image
            annotated = results[0].plot()
            annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
            
            # Display
            axes[idx].imshow(annotated_rgb)
            num_detections = len(results[0].boxes)
            
            # Color code title based on detections
            if num_detections > 0:
                title_color = 'green'
                title = f'{img_path.stem}\n{num_detections} detection(s)'
            else:
                title_color = 'red'
                title = f'{img_path.stem}\nNo detections'
            
            axes[idx].set_title(title, fontsize=11, fontweight='bold', color=title_color)
            axes[idx].axis('off')
            
        except Exception as e:
            print(f"WARNING: Error processing {img_path.name}: {str(e)}")
            axes[idx].text(0.5, 0.5, f'Error: {img_path.stem}', ha='center', va='center')
            axes[idx].axis('off')
    
    plt.tight_layout()
    save_path = config.PLOTS_DIR / 'sample_predictions.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Sample predictions saved to: {save_path}")

In [ ]:
# ============================================================================
# VISUALIZATION 2: Confidence Distribution Analysis
# ============================================================================
print("\n" + "=" * 80)
print("CONFIDENCE SCORE ANALYSIS")
print("=" * 80)

# Get validation images (use from previous cell if available)
val_img_dir = Path(config.DATASET_PATH) / 'images' / 'val'
if 'all_val_images' not in globals() or len(all_val_images) == 0:
    if val_img_dir.exists():
        all_val_images = list(val_img_dir.glob('*.png'))
    else:
        all_val_images = []

# Check if we have images to analyze
if len(all_val_images) == 0:
    print("\nERROR: No validation images found for analysis!")
    print(f"  - Directory: {val_img_dir}")
    print("  - Please check your dataset configuration")
else:
    # Run predictions on all validation images
    all_confidences = []
    all_classes = []
    detection_stats = {'with_detection': 0, 'without_detection': 0}

    print(f"\nAnalyzing {len(all_val_images)} validation images...\n")

    for img_path in tqdm(all_val_images, desc="Processing"):
        try:
            results = best_model.predict(
                source=str(img_path),
                conf=config.CONF_THRESHOLD,
                iou=config.IOU_THRESHOLD,
                imgsz=config.IMG_SIZE,
                device=config.DEVICE,
                verbose=False
            )
            
            if len(results[0].boxes) > 0:
                detection_stats['with_detection'] += 1
                for box in results[0].boxes:
                    all_confidences.append(float(box.conf.cpu()))
                    all_classes.append(int(box.cls.cpu()))
            else:
                detection_stats['without_detection'] += 1
        except Exception as e:
            print(f"\nWARNING: Error processing {img_path.name}: {str(e)}")
            detection_stats['without_detection'] += 1

    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('YOLOv10 - Detection Confidence Analysis', fontsize=16, fontweight='bold')

    # 1. Confidence Distribution
    if all_confidences:
        axes[0, 0].hist(all_confidences, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
        axes[0, 0].axvline(config.CONF_THRESHOLD, color='red', linestyle='--', 
                           label=f'Threshold: {config.CONF_THRESHOLD}', linewidth=2)
        axes[0, 0].set_xlabel('Confidence Score', fontsize=12)
        axes[0, 0].set_ylabel('Frequency', fontsize=12)
        axes[0, 0].set_title(f'Confidence Score Distribution\n(Total Detections: {len(all_confidences)})', fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(alpha=0.3)
    else:
        axes[0, 0].text(0.5, 0.5, 'No Detections Found', ha='center', va='center', fontsize=14, color='red')
        axes[0, 0].set_title('Confidence Score Distribution', fontweight='bold')

    # 2. Detection Statistics
    categories = ['Images with\nDetections', 'Images without\nDetections']
    values = [detection_stats['with_detection'], detection_stats['without_detection']]
    colors = ['#4CAF50', '#f44336']

    axes[0, 1].bar(categories, values, color=colors, edgecolor='black', linewidth=2)
    axes[0, 1].set_ylabel('Number of Images', fontsize=12)
    axes[0, 1].set_title('Detection Coverage Analysis', fontweight='bold')
    for i, v in enumerate(values):
        axes[0, 1].text(i, v + max(values)*0.02 if max(values) > 0 else 0.5, str(v), 
                        ha='center', va='bottom', fontsize=14, fontweight='bold')
    axes[0, 1].grid(axis='y', alpha=0.3)

    # 3. Class Distribution
    if all_classes:
        class_counts = pd.Series(all_classes).value_counts().sort_index()
        class_names = [config.CLASS_NAMES[i] for i in class_counts.index]
        
        axes[1, 0].barh(class_names, class_counts.values, color='coral', edgecolor='black')
        axes[1, 0].set_xlabel('Number of Detections', fontsize=12)
        axes[1, 0].set_title('Detections per Class', fontweight='bold')
        for i, v in enumerate(class_counts.values):
            axes[1, 0].text(v + max(class_counts.values)*0.02, i, str(v), 
                            va='center', fontweight='bold')
        axes[1, 0].grid(axis='x', alpha=0.3)
    else:
        axes[1, 0].text(0.5, 0.5, 'No Detections Found', ha='center', va='center', fontsize=14, color='red')
        axes[1, 0].set_title('Detections per Class', fontweight='bold')

    # 4. Confidence Box Plot by Class
    if all_confidences and all_classes:
        conf_by_class = pd.DataFrame({'Class': all_classes, 'Confidence': all_confidences})
        conf_by_class['Class_Name'] = conf_by_class['Class'].map(lambda x: config.CLASS_NAMES[x])
        
        class_names_unique = conf_by_class['Class_Name'].unique()
        box_data = [conf_by_class[conf_by_class['Class_Name'] == cn]['Confidence'].values 
                    for cn in class_names_unique]
        
        bp = axes[1, 1].boxplot(box_data, labels=class_names_unique, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('lightgreen')
        axes[1, 1].set_ylabel('Confidence Score', fontsize=12)
        axes[1, 1].set_title('Confidence Distribution by Class', fontweight='bold')
        axes[1, 1].grid(axis='y', alpha=0.3)
        axes[1, 1].tick_params(axis='x', rotation=15)
    else:
        axes[1, 1].text(0.5, 0.5, 'No Detections Found', ha='center', va='center', fontsize=14, color='red')
        axes[1, 1].set_title('Confidence Distribution by Class', fontweight='bold')

    plt.tight_layout()
    save_path = config.PLOTS_DIR / 'confidence_analysis.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\nConfidence analysis saved to: {save_path}")
    print(f"\nDetection Summary:")
    
    # Safe division to avoid ZeroDivisionError
    total_images = len(all_val_images)
    if total_images > 0:
        with_pct = (detection_stats['with_detection'] / total_images) * 100
        without_pct = (detection_stats['without_detection'] / total_images) * 100
        print(f"   - Images with detections: {detection_stats['with_detection']} ({with_pct:.1f}%)")
        print(f"   - Images without detections: {detection_stats['without_detection']} ({without_pct:.1f}%)")
    else:
        print(f"   - Images with detections: {detection_stats['with_detection']}")
        print(f"   - Images without detections: {detection_stats['without_detection']}")
    
    print(f"   - Total detections: {len(all_confidences)}")
    
    if all_confidences:
        print(f"   - Average confidence: {np.mean(all_confidences):.4f}")
        print(f"   - Min confidence: {np.min(all_confidences):.4f}")
        print(f"   - Max confidence: {np.max(all_confidences):.4f}")

In [ ]:
# ============================================================================
# VISUALIZATION 3: High-Confidence vs Low-Confidence Detections
# ============================================================================
print("\n" + "=" * 80)
print("HIGH vs LOW CONFIDENCE DETECTIONS")
print("=" * 80)

# Check if we have confidence data from previous cell
if 'all_confidences' not in globals() or len(all_confidences) == 0:
    print("\nWARNING: No detections found in previous analysis.")
    print("Skipping high vs low confidence comparison.")
else:
    # Find images with high and low confidence detections
    high_conf_images = []
    low_conf_images = []
    
    threshold_high = 0.7
    threshold_low = 0.4
    
    # Check if all_val_images exists and is not empty
    if 'all_val_images' not in globals() or len(all_val_images) == 0:
        val_img_dir = Path(config.DATASET_PATH) / 'images' / 'val'
        if val_img_dir.exists():
            all_val_images = list(val_img_dir.glob('*.png'))
        else:
            all_val_images = []
    
    if len(all_val_images) == 0:
        print("\nERROR: No validation images available for comparison")
    else:
        for img_path in all_val_images:
            try:
                results = best_model.predict(
                    source=str(img_path),
                    conf=config.CONF_THRESHOLD,
                    imgsz=config.IMG_SIZE,
                    device=config.DEVICE,
                    verbose=False
                )
                
                if len(results[0].boxes) > 0:
                    max_conf = max([float(box.conf.cpu()) for box in results[0].boxes])
                    if max_conf >= threshold_high:
                        high_conf_images.append((img_path, results, max_conf))
                    elif max_conf <= threshold_low:
                        low_conf_images.append((img_path, results, max_conf))
            except Exception as e:
                continue  # Skip problematic images
        
        # Create comparison visualization
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('YOLOv10 - Confidence Comparison: High vs Low', fontsize=16, fontweight='bold')
        
        # Plot high confidence (top row)
        for idx in range(3):
            if idx < len(high_conf_images):
                img_path, results, conf = high_conf_images[idx]
                annotated = results[0].plot()
                annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
                axes[0, idx].imshow(annotated_rgb)
                axes[0, idx].set_title(f'HIGH CONF: {conf:.3f}\n{img_path.stem}', 
                                      fontsize=10, fontweight='bold', color='green')
            else:
                axes[0, idx].text(0.5, 0.5, 'No High\nConfidence\nDetections', 
                                ha='center', va='center', fontsize=12)
            axes[0, idx].axis('off')
        
        # Plot low confidence (bottom row)
        for idx in range(3):
            if idx < len(low_conf_images):
                img_path, results, conf = low_conf_images[idx]
                annotated = results[0].plot()
                annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
                axes[1, idx].imshow(annotated_rgb)
                axes[1, idx].set_title(f'LOW CONF: {conf:.3f}\n{img_path.stem}', 
                                      fontsize=10, fontweight='bold', color='orange')
            else:
                axes[1, idx].text(0.5, 0.5, 'No Low\nConfidence\nDetections', 
                                ha='center', va='center', fontsize=12)
            axes[1, idx].axis('off')
        
        plt.tight_layout()
        save_path = config.PLOTS_DIR / 'confidence_comparison.png'
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\nConfidence comparison saved to: {save_path}")
        print(f"   - High confidence images (>={threshold_high}): {len(high_conf_images)}")
        print(f"   - Low confidence images (<={threshold_low}): {len(low_conf_images)}")

In [ ]:
# ============================================================================
# VISUALIZATION 4: Training Curves from results.csv
# ============================================================================
print("\n" + "=" * 80)
print("📈 TRAINING CURVES VISUALIZATION")
print("=" * 80)

# Load results CSV
results_csv = config.MODEL_DIR / 'train' / 'results.csv'

if results_csv.exists():
    results_df = pd.read_csv(results_csv)
    results_df.columns = results_df.columns.str.strip()  # Remove whitespace
    
    # Create comprehensive training curves
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    fig.suptitle('YOLOv10 - Training Progress Curves', fontsize=16, fontweight='bold')
    
    epochs = results_df.index + 1
    
    # 1. Loss Curves
    axes[0, 0].plot(epochs, results_df['train/box_loss'], label='Train Box Loss', linewidth=2, marker='o', markersize=3)
    axes[0, 0].plot(epochs, results_df['val/box_loss'], label='Val Box Loss', linewidth=2, marker='s', markersize=3)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Box Loss (Train vs Val)', fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # 2. Classification Loss
    axes[0, 1].plot(epochs, results_df['train/cls_loss'], label='Train Cls Loss', linewidth=2, marker='o', markersize=3, color='orange')
    axes[0, 1].plot(epochs, results_df['val/cls_loss'], label='Val Cls Loss', linewidth=2, marker='s', markersize=3, color='red')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title('Classification Loss (Train vs Val)', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # 3. mAP@0.5
    axes[1, 0].plot(epochs, results_df['metrics/mAP50(B)'], linewidth=2, marker='D', markersize=4, color='green')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('mAP@0.5')
    axes[1, 0].set_title('mAP@0.5 Progress', fontweight='bold')
    axes[1, 0].grid(alpha=0.3)
    axes[1, 0].fill_between(epochs, results_df['metrics/mAP50(B)'], alpha=0.3, color='green')
    
    # 4. mAP@0.5:0.95
    axes[1, 1].plot(epochs, results_df['metrics/mAP50-95(B)'], linewidth=2, marker='D', markersize=4, color='blue')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('mAP@0.5:0.95')
    axes[1, 1].set_title('mAP@0.5:0.95 Progress', fontweight='bold')
    axes[1, 1].grid(alpha=0.3)
    axes[1, 1].fill_between(epochs, results_df['metrics/mAP50-95(B)'], alpha=0.3, color='blue')
    
    # 5. Precision & Recall
    axes[2, 0].plot(epochs, results_df['metrics/precision(B)'], label='Precision', linewidth=2, marker='o', markersize=3, color='purple')
    axes[2, 0].plot(epochs, results_df['metrics/recall(B)'], label='Recall', linewidth=2, marker='s', markersize=3, color='brown')
    axes[2, 0].set_xlabel('Epoch')
    axes[2, 0].set_ylabel('Score')
    axes[2, 0].set_title('Precision & Recall Progress', fontweight='bold')
    axes[2, 0].legend()
    axes[2, 0].grid(alpha=0.3)
    
    # 6. Learning Rate
    axes[2, 1].plot(epochs, results_df['lr/pg0'], label='LR Group 0', linewidth=2, marker='o', markersize=3)
    axes[2, 1].plot(epochs, results_df['lr/pg1'], label='LR Group 1', linewidth=2, marker='s', markersize=3)
    axes[2, 1].plot(epochs, results_df['lr/pg2'], label='LR Group 2', linewidth=2, marker='^', markersize=3)
    axes[2, 1].set_xlabel('Epoch')
    axes[2, 1].set_ylabel('Learning Rate')
    axes[2, 1].set_title('Learning Rate Schedule', fontweight='bold')
    axes[2, 1].legend()
    axes[2, 1].grid(alpha=0.3)
    axes[2, 1].set_yscale('log')
    
    plt.tight_layout()
    save_path = config.PLOTS_DIR / 'training_curves_detailed.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n✅ Training curves saved to: {save_path}")
    
    # Print final metrics
    print(f"\n📊 Final Epoch Metrics:")
    print(f"   • mAP@0.5: {results_df['metrics/mAP50(B)'].iloc[-1]:.4f}")
    print(f"   • mAP@0.5:0.95: {results_df['metrics/mAP50-95(B)'].iloc[-1]:.4f}")
    print(f"   • Precision: {results_df['metrics/precision(B)'].iloc[-1]:.4f}")
    print(f"   • Recall: {results_df['metrics/recall(B)'].iloc[-1]:.4f}")
else:
    print(f"\n⚠️  Results CSV not found at: {results_csv}")

In [ ]:
# Create comprehensive metrics summary
print("=" * 80)
print("📊 YOLOV10 FINAL METRICS SUMMARY")
print("=" * 80)

summary_data = {
    'Metric': [
        'Model',
        'Epochs Trained',
        'Training Time (min)',
        'mAP@0.5',
        'mAP@0.5:0.95',
        'Precision',
        'Recall',
        'F1 Score',
        'Fitness Score',
        'Image Size',
        'Batch Size',
        'Optimizer',
        'Learning Rate',
    ],
    'Value': [
        config.MODEL_NAME,
        config.EPOCHS,
        f"{training_time/60:.2f}",
        f"{float(val_results.box.map50):.4f}",
        f"{float(val_results.box.map):.4f}",
        f"{float(val_results.box.mp):.4f}",
        f"{float(val_results.box.mr):.4f}",
        f"{2 * (float(val_results.box.mp) * float(val_results.box.mr)) / (float(val_results.box.mp) + float(val_results.box.mr) + 1e-6):.4f}",
        f"{float(val_results.fitness):.4f}",
        f"{config.IMG_SIZE}x{config.IMG_SIZE}",
        config.BATCH_SIZE,
        config.OPTIMIZER,
        config.LR0,
    ]
}

summary_df = pd.DataFrame(summary_data)

# Display styled table
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns,
                cellLoc='left', loc='center',
                colWidths=[0.6, 0.4])

table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

# Style header
for i in range(len(summary_df.columns)):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Alternate row colors
for i in range(1, len(summary_df) + 1):
    for j in range(len(summary_df.columns)):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#f0f0f0')

plt.title('YOLOv10 Training & Validation Metrics', fontsize=16, fontweight='bold', pad=20)
save_path = config.PLOTS_DIR / 'metrics_summary.png'
plt.savefig(save_path, dpi=config.DPI, bbox_inches='tight')
plt.show()

print(f"\n✅ Metrics summary saved to: {save_path}")

# Also save as CSV
csv_path = config.RESULTS_DIR / 'metrics_summary.csv'
summary_df.to_csv(csv_path, index=False)
print(f"✅ Metrics CSV saved to: {csv_path}")

## 📋 Section 10: Final Report

In [ ]:
# Generate comprehensive markdown report
report_path = config.RESULTS_DIR / 'yolov10_training_report.md'

report_content = f"""# YOLOv10 Training Report - TBX11K Tuberculosis Detection

**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  
**Student:** Turjo Khan  
**Institution:** East West University  
**Course:** CSE475 - Machine Learning Lab

---

## 🎯 Model Information

- **Model:** {config.MODEL_NAME}
- **Architecture:** YOLOv10 Nano
- **Pretrained Weights:** {config.MODEL_WEIGHTS}
- **Task:** Object Detection (Tuberculosis in Chest X-rays)

---

## 📊 Dataset

- **Dataset:** TBX11K (Balanced)
- **Training Images:** {len(train_images)}
- **Validation Images:** {len(val_images)}
- **Classes:** {config.NUM_CLASSES}
  - Class 0: {config.CLASS_NAMES[0]}
  - Class 1: {config.CLASS_NAMES[1]}
  - Class 2: {config.CLASS_NAMES[2]}
- **Image Size:** {config.IMG_SIZE}x{config.IMG_SIZE}

---

## ⚙️ Training Configuration

### Hyperparameters
- **Epochs:** {config.EPOCHS}
- **Batch Size:** {config.BATCH_SIZE}
- **Optimizer:** {config.OPTIMIZER}
- **Initial Learning Rate:** {config.LR0}
- **Final LR Factor:** {config.LRF}
- **Momentum:** {config.MOMENTUM}
- **Weight Decay:** {config.WEIGHT_DECAY}
- **Warmup Epochs:** {config.WARMUP_EPOCHS}
- **Patience:** {config.PATIENCE}

### Loss Weights
- **Box Loss:** {config.BOX}
- **Class Loss:** {config.CLS}
- **DFL Loss:** {config.DFL}

### Data Augmentation
- **Rotation:** ±{config.DEGREES}°
- **Translation:** {config.TRANSLATE * 100}%
- **Scaling:** ±{config.SCALE * 100}%
- **Shearing:** ±{config.SHEAR}°
- **Horizontal Flip:** {config.FLIPLR * 100}%
- **Mosaic:** {config.MOSAIC * 100}%
- **MixUp:** {config.MIXUP * 100}%
- **Copy-Paste:** {config.COPY_PASTE * 100}%
- **Random Erasing:** {config.ERASING * 100}%
- **HSV Augmentation:** H={config.HSV_H}, S={config.HSV_S}, V={config.HSV_V}

---

## 📈 Training Results

- **Training Time:** {training_time/60:.2f} minutes ({training_time/3600:.2f} hours)
- **Best Model Path:** `{best_model_path}`

---

## 🎯 Validation Metrics

| Metric | Value |
|--------|-------|
| **mAP@0.5** | {float(val_results.box.map50):.4f} |
| **mAP@0.5:0.95** | {float(val_results.box.map):.4f} |
| **Precision** | {float(val_results.box.mp):.4f} |
| **Recall** | {float(val_results.box.mr):.4f} |
| **F1 Score** | {2 * (float(val_results.box.mp) * float(val_results.box.mr)) / (float(val_results.box.mp) + float(val_results.box.mr) + 1e-6):.4f} |
| **Fitness** | {float(val_results.fitness):.4f} |

---

## 📁 Output Files

### Models
- Best weights: `{config.MODEL_DIR / 'train' / 'weights' / 'best.pt'}`
- Last weights: `{config.MODEL_DIR / 'train' / 'weights' / 'last.pt'}`

### Visualizations
- Training curves: `{config.PLOTS_DIR / 'training_curves.png'}`
- Confusion matrix: `{config.PLOTS_DIR / 'confusion_matrix_display.png'}`
- Sample predictions: `{config.PLOTS_DIR / 'sample_predictions.png'}`
- Metrics summary: `{config.PLOTS_DIR / 'metrics_summary.png'}`

### Results
- Metrics JSON: `{config.RESULTS_DIR / 'yolov10_metrics.json'}`
- Metrics CSV: `{config.RESULTS_DIR / 'metrics_summary.csv'}`
- Training CSV: `{config.MODEL_DIR / 'train' / 'results.csv'}`

---

## ✅ Conclusion

YOLOv10 training completed successfully with {config.EPOCHS} epochs. The model achieved:
- **mAP@0.5 of {float(val_results.box.map50):.4f}**
- **Training time of {training_time/60:.2f} minutes**

All visualizations and metrics have been saved for analysis.

---

*Generated automatically by YOLOv10 training notebook*
"""

with open(report_path, 'w') as f:
    f.write(report_content)

print("=" * 80)
print("📋 YOLOV10 TRAINING COMPLETE!")
print("=" * 80)
print(f"\n✅ Final report saved to: {report_path}")
print(f"\n📊 Summary:")
print(f"  • Model: {config.MODEL_NAME}")
print(f"  • Epochs: {config.EPOCHS}")
print(f"  • Training Time: {training_time/60:.2f} min")
print(f"  • mAP@0.5: {float(val_results.box.map50):.4f}")
print(f"  • mAP@0.5:0.95: {float(val_results.box.map):.4f}")
print(f"  • Precision: {float(val_results.box.mp):.4f}")
print(f"  • Recall: {float(val_results.box.mr):.4f}")
print(f"\n💾 All outputs saved to: {config.OUTPUT_DIR}")
print("=" * 80)